In [14]:
import torch.nn as nn

from torch.nn.init import xavier_uniform_

model = nn.Linear(40, 10)

print(torch.sum(model.weight))

xavier_uniform_(model.weight, gain=1.41)

print(torch.sum(model.weight))

tensor(2.8718, grad_fn=<SumBackward0>)
tensor(4.3090, grad_fn=<SumBackward0>)


In [6]:
seconds = 94 * (50000/782)
hours = seconds / 3600

hours

1.6695083830633701

In [117]:
import torchvision
import torchvision.transforms as transforms

########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].

def custom_preprocess(tensor):
    transformed_tensor = 2. * tensor - 1.
    transformed_tensor += torch.rand(*transformed_tensor.shape) / 128.
    return transformed_tensor
    
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        custom_preprocess
    ]
)

trainset = torchvision.datasets.CIFAR10(root='Users/kylesargent/Desktop/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)



Files already downloaded and verified


In [45]:
import logging
import sys
logging.basicConfig(filename=os.path.join('/Users/kylesargent/', 'training.log'), level=logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

NameError: name 'root' is not defined

In [15]:
batch_size = 16
epochs = 10
dis_iters = 1

from cifar10_models import Generator, Discriminator

G = Generator()
D = Discriminator()

g_optim = torch.optim.Adam(G.parameters(), lr=.001)
d_optim = torch.optim.Adam(D.parameters(), lr=.001)

for epoch in range(epochs):
    for i in range(train_data.shape[0] // batch_size):
        z = Variable(sample_z(batch_size))
        x_real = Variable(train_data[batch_size * i : batch_size * (i+1)])
        
        for k in range(dis_iters):
            # train discriminator
            x_fake = G(z)
            dis_fake = D(x_fake.detach())
            dis_real = D(x_real)

            loss = dis_loss(dis_fake, dis_real)
            loss.backward()
            d_optim.step()

            # train generator
            if k==0:
                g_optim.zero_grad()
                dis_fake = D(x_fake)

                loss = gen_loss(dis_fake) 
                loss.backward()
                g_optim.step()
        

ImportError: cannot import name 'Generator'

In [21]:
from cifar10_models import Cifar10Generator
from torch.distributions.normal import Normal
from torch.autograd import Variable

def sample_z(batch_size):
    n = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
    return n.sample((batch_size, 128)).squeeze(2)

G = Cifar10Generator()
z = (sample_z(batch_size))

G(z)

tensor([[[[ 1.3192e-01,  1.2600e-01,  3.0534e-01,  ...,  4.8395e-01,
            4.2447e-01,  3.3980e-01],
          [-9.0867e-02, -3.8691e-01, -9.0201e-02,  ...,  2.7825e-01,
           -4.6404e-02,  1.0383e-01],
          [ 5.2356e-02, -3.8307e-01, -1.0839e-01,  ...,  5.4389e-01,
            2.9140e-01,  3.9991e-01],
          ...,
          [ 5.2455e-01,  6.5654e-01,  6.7763e-01,  ...,  2.7052e-01,
            3.3309e-01,  3.5444e-01],
          [ 3.9760e-01,  4.5689e-01,  4.7463e-01,  ...,  2.4466e-01,
            2.9873e-01,  2.6779e-01],
          [ 1.4250e-01,  1.2857e-01,  2.6162e-01,  ...,  1.4585e-01,
            2.4795e-01, -4.9396e-02]],

         [[-3.5721e-01, -1.5552e-01, -1.9746e-01,  ..., -2.9610e-01,
           -2.7172e-01,  6.7485e-02],
          [-4.7579e-02, -8.0179e-02, -1.6852e-01,  ..., -2.1015e-01,
           -1.8315e-01, -1.6189e-02],
          [ 2.8562e-01,  2.3068e-01,  1.7587e-01,  ...,  9.8998e-02,
           -1.4995e-01, -3.6112e-02],
          ...,
     